In [1]:
import os
import sys

# 获取当前工作目录（Notebook 启动目录）
CURRENT_WORK_DIR = os.getcwd()

# 跳转到上上级目录
ROOT_DIR = os.path.abspath(os.path.join(CURRENT_WORK_DIR, '..', '..','..'))

# 切换工作目录
os.chdir(ROOT_DIR)

# 添加到模块搜索路径
sys.path.append(ROOT_DIR)

print("Working from:", os.getcwd())


Working from: d:\pandalow_playgrounds\project\Toxilen_model


In [2]:
import pandas as pd
from torch import nn
from PIL import Image
from transformers import AutoProcessor, AutoModelForVisualQuestionAnswering

In [7]:
device = 'cuda'
model_name = 'Salesforce/blip2-opt-2.7b'

In [3]:
import json

with open('data\MET\converted_data.json',"r") as f:
    data = json.load(f)

print(data[0])


{'filename': 'Image_(1).jpg', 'original_text': '呵！辣鸡', 'label': 1, 'span': 'Detected Hateful spans: 辣鸡', 'prompt': '<HATE_SPAN> 辣鸡 </HATE_SPAN> 呵！辣鸡'}


<>:3: SyntaxWarning: invalid escape sequence '\M'
<>:3: SyntaxWarning: invalid escape sequence '\M'
C:\Users\zxj00\AppData\Local\Temp\ipykernel_43448\2485267197.py:3: SyntaxWarning: invalid escape sequence '\M'
  with open('data\MET\converted_data.json',"r") as f:


In [4]:
train_df = pd.DataFrame(data)
train_df.head()

,filename,original_text,label,span,prompt
0,Image_(1).jpg,呵！辣鸡,1,Detected Hateful spans: 辣鸡,<HATE_SPAN> 辣鸡 </HATE_SPAN> 呵！辣鸡
1,Image_(10).jpg,我只是一个宝宝却要承受这么多,0,Detected general emotions spans: 承受这么多,<EMO_SPAN> 承受这么多 </EMO_SPAN> 我只是一个宝宝却要承受这么多
2,Image_(100).jpg,吃太撑了鸭,0,Detected general emotions spans: 吃太撑了,<EMO_SPAN> 吃太撑了 </EMO_SPAN> 吃太撑了鸭
3,Image_(1000).jpg,辣,1,Detected general emotions spans: 辣,<EMO_SPAN> 辣 </EMO_SPAN> 辣
4,Image_(1001).jpg,我大概是个辣鸡了,0,Detected Hateful spans: 辣鸡,<HATE_SPAN> 辣鸡 </HATE_SPAN> 我大概是个辣鸡了


In [5]:
file_names = train_df['filename'].to_list()

In [ ]:
processor =  AutoProcessor.from_pretrained(model_name)
model = AutoModelForVisualQuestionAnswering.from_pretrained(model_name).to(device)

model-00001-of-00002.safetensors:   1%|1         | 115M/10.1G [00:00<?, ?B/s]

In [ ]:
def generate_caption(image_path):
    raw_image = Image.open(image_path).convert('RGB')
    inputs = processor(raw_image,return_tensors='pt').to(device)
    out = model.generate(**inputs)
    caption = processor.decode(out[0], skip_special_tokens=True)
    
    return caption

In [ ]:
caption_outputs = []
for image_path in file_names:
    image = rf"data\toxi_mm\images\Image-{image_path[5:]}"
    caption = generate_caption(image_path=image)    
    caption_outputs.append(caption)
    print(caption)

C:\Users\zxj00\AppData\Roaming\Python\Python312\site-packages\transformers\generation\utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


a cartoon character with a capt that reads, ' i ' m ' m ' m ' m
a picture of a small ant ant ant ant ant ant ant ant ant ant ant ant ant ant
a cartoon of a dog sleeping in bed
a little girl is sitting at a table with a toy
a cartoon character with a red hair and a green shirt
a panda with a smile on its face
a cartoon of a man talking to another man
a poster with a bear holding a cup
a man is seen in a video of a woman being rapeded by a man
a man in a white shirt and sunglasses waving
a cat is being fed by a person
a person is washing their hands with soap
the legend of the condor
a man kissing a woman with a rabbit on his shoulder
a cartoon depicting the president and his wife at a table
a col of a man with a banana on his head
a cartoon of a man with a face and a capt that says, ' i ' m '
a man and woman are eating a hamburger and a woman is eating a hamburger
a cat is sitting on the floor next to a pair of sneakers
a woman in uniform is holding a camera
a panda bear sitting on top 

KeyboardInterrupt: 

In [49]:
train_df['captions'] = caption_outputs
train_df.to_csv('data/toxi_mm/train_caption_version.csv')